In [45]:
import numpy as np
#import copy
#from scipy.stats import uniform
#from scipy.stats import norm
from scipy.stats import t
from scipy import integrate

In [46]:
X = np.array([[1,0,0,0,0,83],
              [1,0,0,0,0,85],
              [0,1,0,0,0,84],
              [0,1,0,0,0,85],
              [0,1,0,0,0,85],
              [0,1,0,0,0,86],
              [0,1,0,0,0,86],
              [0,1,0,0,0,87],
              [0,0,1,0,0,86],
              [0,0,1,0,0,87],
              [0,0,1,0,0,87],
              [0,0,1,0,0,87],
              [0,0,1,0,0,88],
              [0,0,1,0,0,88],
              [0,0,1,0,0,88],
              [0,0,1,0,0,88],
              [0,0,1,0,0,88],
              [0,0,1,0,0,89],
              [0,0,1,0,0,90],
              [0,0,0,1,0,89],
              [0,0,0,1,0,90],
              [0,0,0,1,0,90],
              [0,0,0,1,0,91],
              [0,0,0,0,1,90],
              [0,0,0,0,1,92]])

In [47]:
def lin_regr(X):
    N, dim = X.shape[0], X.shape[1] - 1
    
    PSI = X.T[0:dim].T
    Y = X.T[dim].T

    F = np.matmul(PSI.T,PSI)
    F1 = np.linalg.inv(F)
    BETA = np.matmul(np.matmul(F1,PSI.T),Y)
    
    e = Y - np.matmul(PSI,BETA)
    RSS = np.matmul(e.T,e)
    TSS = np.sum((Y-np.mean(Y))**2)
    R = (1 - RSS/TSS)**0.5
    
    PVAL = []
    for i in range(len(BETA)):
        df = N-dim
        delta = BETA[i]*(df)**0.5/(RSS*F1[i][i])**0.5
        
        def func(x):
            return t.pdf(x, df, loc=0, scale=1)
        pval, err = integrate.quad(func, np.abs(delta), np.inf)
        PVAL.append(2*pval)
    
    PVALD = []
    INDS = []
    for i in range(len(BETA)-1):
        for j in range(i+1,len(BETA)):
            INDS.append((i,j))
            df = N-dim
            delta = (BETA[i]-BETA[j])*(df)**0.5/(RSS*(F1[i][i]+F1[j][j]))**0.5
            
            def func(x):
                return t.pdf(x, df, loc=0, scale=1)
            pval, err = integrate.quad(func, np.abs(delta), np.inf)
            PVALD.append(2*pval)
       
    return (BETA, R, PVAL, INDS, PVALD)

In [48]:
def regr_info(X):
    beta,r,pval,inds,pvald = lin_regr(X)
    
    mnoj = [f"{round(beta[0],2)}"]+[f"({round(beta[i+1],2)})*x{i+1}" for i in range(beta.shape[0]-1)]
    urav = " + ".join(mnoj)
    print("Уравнение лин. регрессии:\n"+urav+"\n")
    
    alpha = 0.05
    print("При этом коэфф.:")
    for i in range(len(pval)):
        ne = "не "
        if pval[i]<alpha:
            ne = ""
        print(f"\tb{i} {ne}явл. значимым с p-val = {round(pval[i],4)}")
    print()
        
    print(f"Коэфф. детерм. R = {r}\n")
    
    print("Попарно коэфф.:")
    for i in range(len(pvald)):
        ne = "="
        if pvald[i]<alpha:
            ne = "!="
        print(f"\tb{inds[i][0]} {ne} b{inds[i][1]} с p-val = {round(pvald[i],4)}")
    print()

In [49]:
regr_info(X)

Уравнение лин. регрессии:
84.0 + (85.5)*x1 + (87.82)*x2 + (90.0)*x3 + (91.0)*x4

При этом коэфф.:
	b0 явл. значимым с p-val = 0.0
	b1 явл. значимым с p-val = 0.0
	b2 явл. значимым с p-val = 0.0
	b3 явл. значимым с p-val = 0.0
	b4 явл. значимым с p-val = 0.0

Коэфф. детерм. R = 0.90033663737852

Попарно коэфф.:
	b0 = b1 с p-val = 0.1031
	b0 != b2 с p-val = 0.0002
	b0 != b3 с p-val = 0.0
	b0 != b4 с p-val = 0.0
	b1 != b2 с p-val = 0.0004
	b1 != b3 с p-val = 0.0
	b1 != b4 с p-val = 0.0
	b2 != b3 с p-val = 0.0024
	b2 != b4 с p-val = 0.001
	b3 = b4 с p-val = 0.2958

